**Lista de Imports**

In [1]:
import numpy as np
import time
import pandas as pd
from IPython.display import display
from scipy.optimize import linprog, minimize
import time
from scipy.linalg import solve
from collections import Counter

**Punto 1**

G&L es un pequeño fabricante de equipo y accesorios para golf donde su distribuidor apunta está dispuesto en adquirir en los siguientes tres meses todas las bolsas de golf para dos tipos de públicos específicos: Modelo Estándar y Modelo Deluxe. El director de manufactura estima que durante los siguientes tres meses los tiempos disponibles en manufactura están segementados para 630 horas de tiempo de corte y teñido, 600 de tiempo de costura, 708 horas de tiempo de terminado y 135 horas de tiempo de inspección y empaque para la producción de las bolsas de golf. Como caso de análisis, se busca maximizar las utilidades de G&L.

**Variables de Decisión**

$X_{1}:$ Cantidad de bolsas del Modelo Estándar

$X_{2}:$ Cantidad de bolsas del Modelo Deluxe


**Función Objetivo**

$max  F(x_{1},x_{2}):$ $U_{Me} \cdot X_{1}$ + $U_{Md} \cdot X_{2} $ 

*Para el Problema Primal*

$min F(x_{1},x_{2}):$ $-U_{Me} \cdot X_{1}$ - $U_{Md} \cdot X_{2}$ 

Donde,

$U_{Me}:$ Utilidad del bolso de Modelo Estándar ($7)

$U_{Md}:$ Utilidad del bolso de Modelo Deluxe ($9)


**Planteamiento de Restricciones**

$\frac{7}{10} \cdot X_{1} + X_{2} \leq T_{CyT} \longmapsto -\frac{7}{10} \cdot X_{1} - X_{2} \geq -T_{CyT}$ 

$\frac{1}{2} \cdot X_{1} + \frac{5}{6}X_{2} \leq T_{Co} \longmapsto -\frac{1}{2} \cdot X_{1} - \frac{5}{6}X_{2} \geq -T_{Co}$ 

$ X_{1} + \frac{2}{3} \cdot X_{2} \leq T_{Te} \longmapsto -X_{1} - \frac{2}{3} \cdot X_{2} \geq -T_{Te}$ 

$ \frac{1}{10} \cdot X_{1} + \frac{1}{4} \cdot X_{2} \leq T_{IyE} \longmapsto -\frac{1}{10} \cdot X_{1} - \frac{1}{4} \cdot X_{2} \geq -T_{IyE}$ 

$X_{1}, X_{2} \geq 0$

Donde,

$T_{CyT}:$ Tiempo máximo de la planta para el Corte y Teñido (630h)

$T_{Co}:$ Tiempo máximo de la planta para la Costura (600h)

$T_{Te}:$ Tiempo máximo de la planta para el Terminado (708h)

$T_{IyE}:$ Tiempo máximo de la planta para la Inspección y Empaque (135h)


**Solución del Problema Primal**

Se hace el planteamiento como un problema LP de minimazación, es decir:


In [2]:
#Utilidad del bolso: Estándar, Deluxe ($)
utilidad = np.array([[7, 9],
                   [7, 0],
                   [0, 9]])

#Tiempos de Corte y Teñido: Estándar, Deluxe, Máximo [h]
tiempos_CyT = np.array([7/10, 1, 630]) 

#Tiempos de Costura: Estándar, Deluxe, Máximo [h]
tiempos_Co = np.array([1/2, 5/6, 600]) 

#Tiempos de Terminado: Estándar, Deluxe, Máximo [h]
tiempos_Te = np.array([1, 2/3, 708]) 

#Tiempos de Terminado: Estándar, Deluxe, Máximo [h]
tiempos_IyE = np.array([1/10, 1/4, 135]) 


#Matriz de restricciones
A = np.array([[tiempos_CyT[0], tiempos_CyT[1]],
             [tiempos_Co[0], tiempos_Co[1]],
             [tiempos_Te[0], tiempos_Te[1]],
             [tiempos_IyE[0], tiempos_IyE[1]],
             ])

#Matriz coeficientes inequalidades
b = np.array([tiempos_CyT[2], tiempos_Co[2], tiempos_Te[2], tiempos_IyE[2]])

#Restricción de variables no negativas
bounds = [(  0,    None)]*2

#Valor función objetivo y solución de las variables básicas
val_p1 = []
sol_p1 = []

for i in range(utilidad.shape[0]):
    #Función objetivo
    c = utilidad[i]
    
    #Se transforma el problema a uno de minimización
    res_p1 = linprog(-1*c, A_ub=A, b_ub=b, bounds=bounds, method='revised simplex', options={"disp": False})

    #Se agregan los resultados en la lista
    val_p1.append(round(res_p1.fun*-1, ndigits=2))
    sol_p1.append(np.around(res_p1.x, decimals=2))

#Visualización de los datos
data_p1 = {"Tipo": ['Normal', 'B. Estándar', 'B. Deluxe'],
          "Valor [$]": val_p1,
          "Solución [X1, X2]": sol_p1}

print('En la siguiente tabla se recopila el resultado de la función objetivo y solución de las variables básicas:')
df1 = pd.DataFrame(data_p1)
df1.head()

En la siguiente tabla se recopila el resultado de la función objetivo y solución de las variables básicas:


,Tipo,Valor [$],"Solución [X1, X2]"
0,Normal,6048.0,"[540.0, 252.0]"
1,B. Estándar,4956.0,"[708.0, 0.0]"
2,B. Deluxe,4860.0,"[0.0, 540.0]"


Tal y como se evidencia anteriormente, la mejor estrategia resulta ser la primera ya que es la que presenta una mayor utilidad. En cuanto a las otras dos, se comprueba que se garantiza unicamente la manufactura de un solo producto, sea estándat o deluxe.

**Punto 2**

*Planteamiento del Problema Dual*

**Función Objetivo**

max $G(\lambda_{1}, \lambda_{2}, \lambda_{3}, \lambda_{4}):$ $-T_{CyT} \cdot \lambda_{1} - T_{Co} \cdot \lambda_{2} - T_{Te} \cdot \lambda_{3} - T_{IyE} \cdot \lambda_{4}$ 

Donde,

$T_{CyT}:$ Tiempo máximo de la planta para el Corte y Teñido (630h)

$T_{Co}:$ Tiempo máximo de la planta para la Costura (600h)

$T_{Te}:$ Tiempo máximo de la planta para el Terminado (708h)

$T_{IyE}:$ Tiempo máximo de la planta para la Inspección y Empaque (135h)

**Ecuaciones de Restricción**

$-\frac{7}{10} \cdot \lambda_{1} - \frac{1}{2} \cdot \lambda_{2} - \lambda_{3} - \frac{1}{10} \cdot \lambda_{4} \leq -U_{Me}$

$-\lambda_{1} - \frac{5}{6} \cdot \lambda_{2} - \frac{2}{3} \cdot \lambda_{3} - \frac{1}{4} \cdot \lambda_{4} \leq -U_{Md}$

$\lambda_{1}, \lambda_{2}, \lambda_{3}, \lambda_{4} \geq 0$ 

Donde,

$U_{Me}:$ Utilidad del bolso de Modelo Estándar ($7)

$U_{Md}:$ Utilidad del bolso de Modelo Deluxe ($9)


* **Caso 1:** *Aumentar la utilidad de la bolsa Deluxe a \$18 manteniendo la utilidad de la bolsa Estandar*

$maxG(\lambda_{1}, \lambda_{2}, \lambda_{3}, \lambda_{4}): -T_{CyT} \cdot \lambda_{1} - T_{Co} \cdot \lambda_{2} - T_{Te} \cdot \lambda_{3} - T_{IyE} \cdot \lambda_{4}$

S.A

$-\frac{7}{10} \cdot \lambda_{1} - \frac{1}{2} \cdot \lambda_{2} - \lambda_{3} - \frac{1}{10} \cdot \lambda_{4} \leq -7$

$-\lambda_{1} - \frac{5}{6} \cdot \lambda_{2} - \frac{2}{3} \cdot \lambda_{3} - \frac{1}{4} \cdot \lambda_{4} \leq -18$

$\lambda_{1}, \lambda_{2}, \lambda_{3}, \lambda_{4} \geq 0$ 



* **Caso 2:** *Aumentar la utilidad de la bolsa Estandar a \$14 manteniendo la utilidad de la bolsa Deluxe*

$maxG(\lambda_{1}, \lambda_{2}, \lambda_{3}, \lambda_{4}): -T_{CyT} \cdot \lambda_{1} - T_{Co} \cdot \lambda_{2} - T_{Te} \cdot \lambda_{3} - T_{IyE} \cdot \lambda_{4}$


$-\frac{7}{10} \cdot \lambda_{1} - \frac{1}{2} \cdot \lambda_{2} - \lambda_{3} - \frac{1}{10} \cdot \lambda_{4} \leq -14$

$-\lambda_{1} - \frac{5}{6} \cdot \lambda_{2} - \frac{2}{3} \cdot \lambda_{3} - \frac{1}{4} \cdot \lambda_{4} \leq -9$

$\lambda_{1}, \lambda_{2}, \lambda_{3}, \lambda_{4} \geq 0$ 

* **Caso 3:** *Aumentar el trabajo disponible en un 70%*

$maxG(\lambda_{1}, \lambda_{2}, \lambda_{3}, \lambda_{4}): 1.7 \cdot (-T_{CyT} \cdot \lambda_{1} - T_{Co} \cdot \lambda_{2} - T_{Te} \cdot \lambda_{3} - T_{IyE} \cdot \lambda_{4})$

S.A

$-\frac{7}{10} \cdot \lambda_{1} - \frac{1}{2} \cdot \lambda_{2} - \lambda_{3} - \frac{1}{10} \cdot \lambda_{4} \leq -7$

$-\lambda_{1} - \frac{5}{6} \cdot \lambda_{2} - \frac{2}{3} \cdot \lambda_{3} - \frac{1}{4} \cdot \lambda_{4} \leq -9$

$\lambda_{1}, \lambda_{2}, \lambda_{3}, \lambda_{4} \geq 0$ 

In [35]:
#Ecuaciones de restricción: Problema Primal y Dual (Caso 1, Caso 2, Caso 3)
A_p = A
A_d = -1*np.transpose(A)

#Matriz coeficientes: Problema Primal y Dual (Caso 1, Caso 2, Caso 3)
b_p = np.array([b, b, 1.7*b])

b_d = -1*np.array([[7, 18],
                [14, 9],
                [7, 9]])

#Función costo: Primal y Dual (Caso 1, Caso 2, Caso 3)
c_p = b_d
c_d = -1*b_p

#Valor función objetivo y solución de las variables básicas: Primal, Dual
val = []

sol_p = []
sol_d = []

for i in range(b_d.shape[0]):
    #Se transforma el problema a uno de minimización
    res_p = linprog(c_p[i], A_ub=A_p, b_ub=b_p[i], method='revised simplex', options={"disp": False})
    res_d = linprog(-1*c_d[i], A_ub=A_d, b_ub=b_d[i], method='revised simplex', options={"disp": False})
    
    #Se agregan los resultados en la lista
    val_p = round(res_p.fun*-1, ndigits=2)
    val_d = round(res_d.fun, ndigits=2)
    val.append([val_p, val_d])
    
    sol_p.append(np.around(res_p.x, decimals=2))
    sol_d.append(np.around(res_d.x, decimals=2))
    
data_dp = {"Punto": ['a)', 'b)', 'c)'],
         "Valor (Primal, Dual) [$]": val,
         "S. Primal": sol_p, 
         "S. Dual": sol_d}

print('En la siguiente tabla se recopila el resultado de la función objetivo y solución de las variables básicas para el Problema Primal y Dual:')
dps = pd.DataFrame(data_dp)
dps.head()



En la siguiente tabla se recopila el resultado de la función objetivo y solución de las variables básicas para el Problema Primal y Dual:


,Punto,"Valor (Primal, Dual) [$]",S. Primal,S. Dual
0,a),"[9720.0, 9720.0]","[0.0, 540.0]","[0.0, 0.0, 0.0, 72.0]"
1,b),"[9912.0, 9912.0]","[708.0, 0.0]","[0.0, 0.0, 14.0, 0.0]"
2,c),"[10281.6, 10281.6]","[918.0, 428.4]","[8.12, 0.0, 1.31, 0.0]"


Como se puede ver la mejor estrategia es la C) ya que es la que presenta una mayor utilidad frente a las demás opciones.

**Punto 3**

Se desarrolla una función que recibe como parámetros la información del Problema Primal y su respectiva solución en cuanto al Problema Dual. Esta permite al usuario hallar la solución del problema primal sin la necesidad de una fase adicional de optimización.

In [81]:
def sol_holgura_complementaria(A, b, f, res):
    reg = []
    q = np.transpose(res)*b
    Ls = np.size(res)
    for i in range(Ls):
        if res[i] > 0:
            reg.append(i)
    A_in = []
    b_in = []
    for i in reg:
        A_in.append(A[i])
        b_in.append(b[i])
    A_in.append(f)
    b_in.append(q)
    sol_p = solve(A_in,b_in)
    return sol_p